# Advanced Querying Mongo

**⚠️ IMPORTANT: Limit your print to avoid infinite scrolling. Otherwise your
code will be lost between print lines. If working with lists do:**

```python
list(collection.find(query))[:5] #or a reasonably low number
```

Importing libraries and setting up connection

In [ ]:
import pandas as pd
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017")
client.list_database_names()
db = client["Ironhack"]
c = db.get_collection("companies")

In [249]:
filter = {}
projection = {'deadpooled_year': 1,"_id":0,}
list(c.find(filter,projection))


[{'deadpooled_year': 1},
 {'deadpooled_year': 2},
 {'deadpooled_year': 3},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 {'deadpooled_year': 2008},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 {'deadpooled_year': 2013},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 {'deadpooled_year': 2008},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 {'deadpooled_year': None},
 

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [ ]:
list(c.find({"name":"Babelgum"}, projection = {"_id":0, "name": 1}))[:10]


### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [ ]:
filter_1 = {"number_of_employees":{"$gte": 5000}}
projection = {"name": 1, "number_of_employees":1, "_id":0}

a = list(c.find(filter_1, projection).sort("number_of_employees", -1).limit(20))
a

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [ ]:
filter_1 = {"founded_year":{"$gte": 2000, "$lte": 2005}}
projection = {"name": 1, "founded_year":1, "_id":0}

a = list(c.find(filter_1, projection).sort("founded_year", -1))
a

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [ ]:
filter_1 = {"founded_year":{"$lte": 2010}, "ipo.valuation_amount":{"$gte":100000000}}
projection = {"name": 1, "ipo":1, "_id":0}

a = list(c.find(filter_1, projection).sort("founded_year", -1))
a

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [ ]:
filter_1 = {"number_of_employees":{"$lte": 1000},"founded_year":{"$lte": 2005} }
projection = {"name": 1, "number_of_employees":1,"founded_year":1, "_id":0}

a = list(c.find(filter_1, projection).sort("number_of_employees", -1).limit(10))
a

### 6. All the companies that don't include the `partners` field.

In [ ]:
filter_1 = {"partners": { "$exists" : "False" }}
projection = {"name": 1, "number_of_employees":1,"partners":1, "_id":0}

a = list(c.find(filter_1, projection).limit(10))
a

### 7. All the companies that have a null type of value on the `category_code` field.

In [ ]:
filter_1 = {"category_code": None}
projection = {"name": 1, "number_of_employees":1,"category_code":1, "_id":0}

a = list(c.find(filter_1, projection).limit(10))
a

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [ ]:
filter_1 = {"number_of_employees":{"$gte": 100,"$lte": 1000 }}
projection = {"name": 1, "number_of_employees":1, "_id":0}

a = list(c.find(filter_1, projection).sort("number_of_employees", -1).limit(20))
a

### 9. Order all the companies by their IPO price in a descending order.

In [ ]:
filter_1 = {} # Empty dict means * in sql
projection = {"name": 1, "ipo.valuation_amount":1, "_id":0}
a = list(c.find(filter_1,projection).sort("ipo", -1).limit(10))
a

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [ ]:
filter_1 = {}
projection = {"name": 1, "number_of_employees":1, "_id":0}

a = list(c.find(filter_1, projection).sort("number_of_employees", -1).limit(10))
a

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [ ]:
filter_1 = {"founded_month":{"$gt": 6}}
projection = {"name": 1, "founded_year":1, "_id":0}

a = list(c.find(filter_1, projection).sort("founded_year", -1).limit(1000))
a

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.000

In [ ]:
filter_1 = {"founded_year":{"$lte": 2000}, "acquisition.price_amount":{"$gte":10000000}}
projection = {"name": 1, "acquisition.price_amount":1, "_id":0}
a = list(c.find(filter_1, projection).sort("acquisition.price_amount", -1).limit(10))
pd.DataFrame(a)



### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [ ]:
filter_1 = {"acquisition.acquired_year":{"$gte": 2010}}
projection = {"name": 1, "acquisition.price_amount":1, "_id":0}
a = list(c.find(filter_1, projection).sort("acquisition.price_amount", -1).limit(10))
pd.DataFrame(a)

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [207]:
filter_1 = {}
projection = {"name": 1, "founded_year":1, "_id":0}

a = list(c.find(filter_1, projection).sort("founded_year", -1).limit(10))
pd.DataFrame(a)

,name,founded_year
0,Wamba,2013
1,Gimigo,2013
2,Clowdy,2013
3,Fixya,2013
4,Fluc,2013
5,SEOGroup,2013
6,Pikk,2013
7,WhosCall,2013
8,iBazar,2013
9,Advaliant,2013


### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [211]:
filter_1 = {"founded_day":{"$lte": 7}}
projection = {"name": 1, "founded_day":1, "_id":0}
a = list(c.find(filter_1, projection).sort("founded_day", -1))
pd.DataFrame(a)

,name,founded_day
0,Meetup,7
1,Google,7
2,Jive Software,7
3,Rakuten,7
4,Driftr,7
...,...,...
3216,Gambolio,1
3217,SEO Sumo,1
3218,Kicker Films,1
3219,Getyoo,1


### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [215]:
filter_1 = {"category_code": "web","number_of_employees":{"$gte": 4000} }
projection = {"name": 1, "number_of_employees":1,"category_code":1, "_id":0}

a = list(c.find(filter_1, projection).limit(10).sort("number_of_employees",1))
pd.DataFrame(a)

,name,category_code,number_of_employees
0,Expedia,web,4400
1,AOL,web,8000
2,Webkinz,web,8657
3,Rakuten,web,10000
4,Los Angeles Times Media Group,web,10000
5,Groupon,web,10000
6,Yahoo!,web,13600
7,eBay,web,15000
8,Experian,web,15500


### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [226]:
filter_1 = {"acquisition.price_currency_code":"EUR", "acquisition.price_amount":{"$gte":10000000}}
projection = {"name": 1,  "acquisition.price_amount":1, "_id":0}
a = list(c.find(filter_1, projection).sort("acquisition.price_amount", -1).limit(10))
pd.DataFrame(a)

,name,acquisition
0,Apertio,{'price_amount': 140000000}
1,Webedia,{'price_amount': 70000000}
2,Tuenti Technologies,{'price_amount': 70000000}
3,BioMed Central,{'price_amount': 43400000}
4,Greenfield Online,{'price_amount': 40000000}
5,ZYB,{'price_amount': 31500000}
6,Wayfinder,{'price_amount': 24000000}


### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [233]:
filter_1 = {"acquisition.acquired_month":{"$lte": 3.3333}}
projection = {"name": 1, "acquisition":1, "_id":0}
a = list(c.find(filter_1, projection).sort("acquisition.acquired_month", -1).limit(10))
a

[{'name': 'Flickr',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': 'cash_and_stock',
   'source_url': 'http://news.cnet.com/Yahoo-buys-photo-sharing-site-Flickr/2100-1038_3-5627640.html?tag=mncol;txt',
   'source_description': 'CNET',
   'acquired_year': 2005,
   'acquired_month': 3,
   'acquired_day': None,
   'acquiring_company': {'name': 'Yahoo!', 'permalink': 'yahoo'}}},
 {'name': 'Pluck',
  'acquisition': {'price_amount': 75000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://www.techcrunch.com/2008/03/04/demand-media-buys-pluck-for-50-million-to-60-million/',
   'source_description': 'Demand Media Buys Pluck for $75 million',
   'acquired_year': 2008,
   'acquired_month': 3,
   'acquired_day': 4,
   'acquiring_company': {'name': 'Demand Media', 'permalink': 'demandmedia'}}},
 {'name': 'Sconex',
  'acquisition': {'price_amount': 8700000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'sourc

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [237]:
filter_1 = {"founded_year":{"$gte": 2000, "$lte": 2010},"acquisition.acquired_year":{"$lt": 2011}}
projection = {"name": 1, "founded_year":1,"acquisition.acquired_year":1, "_id":0}

a = list(c.find(filter_1, projection).sort("founded_year", -1))
pd.DataFrame(a)

,name,founded_year,acquisition
0,Vusion,2009,{'acquired_year': 2009}
1,Digimix,2009,{'acquired_year': 2009}
2,About Me!,2009,{'acquired_year': 2007}
3,Divvyshot,2009,{'acquired_year': 2010}
4,Cloudkick,2009,{'acquired_year': 2010}
...,...,...,...
708,Skywire Software,2000,{'acquired_year': 2008}
709,Sirific Wireless,2000,{'acquired_year': 2009}
710,LogicLibrary,2000,{'acquired_year': 2008}
711,EnterSys Group,2000,{'acquired_year': 2009}


### 20. All the companies that have been 'deadpooled' after the third year.

In [253]:
filter = {"founded_year":{"gte":0},"Deadpooled_year": {}}
projection = {'deadpooled_year': 1,"_id":0,}
a = list(c.find(filter,projection))
a

[]

In [ ]:
filter_1 = {"founded_year":{"$gte": 2000, "$lte": 2010},"acquisition.acquired_year":{"$lt": 2011}}
projection = {"name": 1, "founded_year":1,"acquisition.acquired_year":1, "_id":0}

a = list(c.find(filter_1, projection).sort("founded_year", -1))
pd.DataFrame(a)

**⚠️ Did you do this?**

```python
list(collection.find(query))[:5] #or a reasonably low number
```